In [1]:
#First we upload our packages
import arcpy
from arcpy import env  
from arcpy.sa import *
import pandas as pd
import geopandas as gpd
import numpy as np
import random
import psycopg2
from shapely.geometry import Point, Polygon

In [3]:
#Next is to read in the CSV of our September 2023 data as dataframe
csv_file_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\temp_data.csv"
temp_data = pd.read_csv(csv_file_path)

In [4]:
#Now let's remove any cells with NA values
temp_data.dropna(inplace=True)

In [5]:
# Then we must convert numeric columns to appropriate data types
numeric_cols = ['high_F', 'low_F', 'precip', 'snow_inch', 'snowd_inch']
temp_data[numeric_cols] = temp_data[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [6]:
#This data also needs coordinate info for each point
#We will use a Mesonet URL to gather reporting station location
url = "https://mesonet.agron.iastate.edu/sites/networks.php?network=MN_COOP&format=csv&nohtml=on"

# Read the CSV data from the URL into a data frame
loc_df = pd.read_csv(url)

In [7]:
# Merge the "lat" and "lon" columns from df based on the matching "nwsli" and "stid" columns
temp_data = pd.merge(temp_data, loc_df[['stid', 'lat', 'lon']], how='left', left_on='nwsli', right_on='stid')

# Drop the redundant "stid" column
temp_data.drop(columns=['stid'], inplace=True)

# Display the updated precip_data data frame
print(temp_data)

      nwsli        date      time  ...  snowd_inch      lat      lon
0     NHPM5  2023-09-01  11:00 PM  ...         0.0  45.0100 -93.3792
1     LMBM5  2023-09-01   7:00 AM  ...         0.0  44.2400 -95.3200
2     LCHM5  2023-09-01   8:00 AM  ...         0.0  45.1279 -94.5348
3     PKGM5  2023-09-01   8:00 AM  ...         0.0  47.2500 -93.5900
4     CLDM5  2023-09-01   6:00 AM  ...         0.0  43.6309 -91.5027
...     ...         ...       ...  ...         ...      ...      ...
3480  KABM5  2023-09-30   7:00 AM  ...         0.0  46.4456 -93.0283
3481  CLDM5  2023-09-30   6:00 AM  ...         0.0  43.6309 -91.5027
3482  NHPM5  2023-09-30  11:00 PM  ...         0.0  45.0100 -93.3792
3483  PKGM5  2023-09-30   8:00 AM  ...         0.0  47.2500 -93.5900
3484  LMBM5  2023-09-30   7:00 AM  ...         0.0  44.2400 -95.3200

[3485 rows x 10 columns]


In [8]:
# Define Minnesota boundary box
minnesota_boundary = Polygon([( -97.5, 43.0), (-89.0, 43.0), (-89.0, 49.5), (-97.5, 49.5)])

# Check if the data falls within the Minnesota boundary
temp_data['Coordinates'] = list(zip(temp_data.lon, temp_data.lat))
temp_data['Coordinates'] = temp_data['Coordinates'].apply(Point)
gdf = gpd.GeoDataFrame(temp_data, geometry='Coordinates')

within_minnesota = gdf[gdf.geometry.within(minnesota_boundary)]
if len(within_minnesota) == 0:
    print("No data falls within Minnesota boundary.")
else:
    print("Data falls within Minnesota boundary.")

Data falls within Minnesota boundary.


In [9]:
# Convert the "Date" column to datetime type
temp_data['date'] = pd.to_datetime(temp_data['date'])

# Check the type of the "Date" column after conversion
date_column_type = temp_data['date'].dtype

print("Type of 'date' column after conversion:", date_column_type)

Type of 'date' column after conversion: datetime64[ns]


In [10]:
#Now we need to choose the date which we will interpolate for
input_date = input("Enter a date (YYYY-MM-DD format): ")

# Convert input string to datetime object
input_date = pd.to_datetime(input_date)

# Filter DataFrame based on the input date
temp_data = temp_data[temp_data['date'] == input_date]

# Display the filtered DataFrame
print(temp_data)

Enter a date (YYYY-MM-DD format): 2023-09-09
      nwsli       date     time  ...      lat      lon               Coordinates
935   RSMM5 2023-09-09  8:00 AM  ...  44.7178 -93.0975  POINT (-93.0975 44.7178)
936   MIAM5 2023-09-09  6:00 PM  ...  45.1219 -95.9269  POINT (-95.9269 45.1219)
937   GNFM5 2023-09-09  8:00 AM  ...  48.1667 -90.8875  POINT (-90.8875 48.1667)
938   MABM5 2023-09-09  7:00 AM  ...  43.5241 -91.7616  POINT (-91.7616 43.5241)
939   HSTM5 2023-09-09  5:00 AM  ...  44.7597 -92.8689  POINT (-92.8689 44.7597)
...     ...        ...      ...  ...      ...      ...                       ...
1045  GLDM5 2023-09-09  9:00 AM  ...  44.5565 -94.2207  POINT (-94.2207 44.5565)
1046  WASM5 2023-09-09  8:00 AM  ...  44.0707 -93.5264  POINT (-93.5264 44.0707)
1047  TOWM5 2023-09-09  7:00 AM  ...  47.7553 -92.2858  POINT (-92.2858 47.7553)
1048  CSLM5 2023-09-09  8:00 AM  ...  47.3847 -94.6147  POINT (-94.6147 47.3847)
1049  AGAM5 2023-09-09  8:00 AM  ...  48.3000 -95.9833     POINT

In [11]:
# This cell will calculate average temperature for each row
temp_data['avg_temp'] = (temp_data['high_F'] + temp_data['low_F']) / 2

In [12]:
# Then these next cells will calculate evapotranspiration
# First it defines the constant for the Hargreaves method
constant = 0.0023

# Convert 'date' column to datetime object
temp_data['date'] = pd.to_datetime(temp_data['date'])

In [13]:
# Then this cell calculates ET using the Hargreaves method for each row
ET0_values = []
for index, row in temp_data.iterrows():
    avg_temperature = row['avg_temp']
    temperature_range = row['high_F'] - row['low_F']
    
    # Calculate the day of the year
    day_of_year = row['date'].dayofyear
    
    # Calculate ET0 using the Hargreaves method
    ET0 = constant * (avg_temperature + 17.8) * (temperature_range ** 0.5) * (1.0 + 0.033 * np.sin(np.deg2rad(360 * (day_of_year - 81) / 365)))
    
    ET0_values.append(ET0)

# Add the calculated ET0 values as a new column to the DataFrame
temp_data['ET'] = ET0_values

# Display the updated DataFrame
print(temp_data)


      nwsli       date     time  ...               Coordinates  avg_temp        ET
935   RSMM5 2023-09-09  8:00 AM  ...  POINT (-93.0975 44.7178)      63.0  1.024494
936   MIAM5 2023-09-09  6:00 PM  ...  POINT (-95.9269 45.1219)      70.5  1.100771
937   GNFM5 2023-09-09  8:00 AM  ...  POINT (-90.8875 48.1667)      57.5  0.718715
938   MABM5 2023-09-09  7:00 AM  ...  POINT (-91.7616 43.5241)      56.5  0.988060
939   HSTM5 2023-09-09  5:00 AM  ...  POINT (-92.8689 44.7597)      64.0  0.927676
...     ...        ...      ...  ...                       ...       ...       ...
1045  GLDM5 2023-09-09  9:00 AM  ...  POINT (-94.2207 44.5565)      67.5  0.947001
1046  WASM5 2023-09-09  8:00 AM  ...  POINT (-93.5264 44.0707)      64.0  1.002005
1047  TOWM5 2023-09-09  7:00 AM  ...  POINT (-92.2858 47.7553)      59.5  0.858185
1048  CSLM5 2023-09-09  8:00 AM  ...  POINT (-94.6147 47.3847)      64.0  0.846848
1049  AGAM5 2023-09-09  8:00 AM  ...     POINT (-95.9833 48.3)      61.5  0.800179

[11

In [16]:
# Then we need to create a feature class for our points
output_fc = 'Points'

# Create a new feature class
arcpy.management.CreateFeatureclass(
    arcpy.env.workspace,
    output_fc,
    'POINT',
    spatial_reference=arcpy.SpatialReference(4326)  # WGS84 Geographic Coordinate System
)

# Check the data type of the 'ET' column in the DataFrame
et_dtype = temp_data['ET'].dtype

# Add field to store ET data, ensuring correct data type
if et_dtype == 'float64':
    arcpy.management.AddField(output_fc, 'ET', 'FLOAT')
else:
    arcpy.management.AddField(output_fc, 'ET', 'DOUBLE')

# Add 'date' and 'nwsli' fields
arcpy.management.AddField(output_fc, 'date', 'DATE')
arcpy.management.AddField(output_fc, 'nwsli', 'TEXT')



# Open an insert cursor
with arcpy.da.InsertCursor(output_fc, ['SHAPE@XY', 'ET', 'date', 'nwsli']) as cursor:
    # Iterate over each row in the DataFrame
    for index, row in temp_data.iterrows():
        # Extract lat, lon, ET, date, and nwsli values
        lat = row['lat']
        lon = row['lon']
        date = row['date']
        ET = row['ET']
        nwsli = row['nwsli']
        
        # Create a point geometry
        point = arcpy.Point(lon, lat)
        point_geometry = arcpy.PointGeometry(point)
        
        # Insert the point feature with the ET, date, and nwsli values
        cursor.insertRow([point_geometry, ET, date, nwsli])

print(f"Feature class '{output_fc}' created successfully.")


Feature class 'Points' created successfully.


In [17]:
#This performs the IDW interpolation
outIDW = Idw("Points.shp", "ET")
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\IDW_ETPoints.tif"
outIDW.save(output_path)

In [18]:
#This performs the ordinary kriging interpolation
outKriging = Kriging("Points.shp", "ET", KrigingModelOrdinary("SPHERICAL", 0.021507), 0.0215068000000001)
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\Kriging_ETPoints.tif"
outKriging.save(output_path)

In [19]:
#This performs the universal kriging interpolation
outKriging = Kriging("Points.shp", "ET", KrigingModelUniversal("SPHERICAL", 0.021507), 0.0215068000000001)
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\Univ_Kriging_ETPoints.tif"
outKriging.save(output_path)

In [20]:
#This performs the resampling of the tifs to reduce the number of points needed
arcpy.management.Resample(
    in_raster=r"Idw_ETPoints.tif",
    out_raster=r"Idw_ET_Resample",
    cell_size="0.2 0.2",
    resampling_type="NEAREST"
)

arcpy.management.Resample(
    in_raster=r"Kriging_ETPoints.tif",
    out_raster=r"Kriging_ET_Resample",
    cell_size="0.2 0.2",
    resampling_type="NEAREST"
)

arcpy.management.Resample(
    in_raster=r"Univ_Kriging_ETPoints.tif",
    out_raster=r"Univ_Kriging_ET_Resample",
    cell_size="0.2 0.2",
    resampling_type="NEAREST"
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5572 Final\\GIS 5572 Final.gdb\\Univ_Kriging_ET_Resample'>

In [21]:
#This converts the raster to points
arcpy.conversion.RasterToPoint(
    in_raster=r"Idw_ET_Resample",
    out_point_features=r"Idw_ET_Point",
    raster_field="value"
)

arcpy.conversion.RasterToPoint(
    in_raster=r"Kriging_ET_Resample",
    out_point_features=r"Kriging_ET_Point",
    raster_field="value"
)

arcpy.conversion.RasterToPoint(
    in_raster=r"Univ_Kriging_ET_Resample",
    out_point_features=r"Univ_Kriging_ET_Point",
    raster_field="value"
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5572 Final\\GIS 5572 Final.gdb\\Univ_Kriging_ET_Point'>

In [ ]:
# QC
# exploratory Interpolation
arcpy.ga.ExploratoryInterpolation(
    in_features="Points",
    value_field="ET",
    out_cv_table=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\ExploratoryInterpolation1",
    out_geostat_layer=None,
    interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)



In [25]:
#This will export the IDW points to a PostGIS database
arcpy.conversion.ExportFeatures(
    in_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\Idw_ET_Point",
    out_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\PostgreSQL-34-final_project(postgres).sde\final_project.postgres.et_data"
)
print("Export complete")

Export complete
